# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [345]:
1+1

2

In [346]:
!cat publications.json

[
    {
        "title": "Unsupervised text augmentation using Pre-trained Paraphrase Generation",
        "pub_date": "2023-09-01", 
        "url_slug": "grok-paper", 
        "paper_url": "",
        "teaser_url": "",
        "venue": "(Preprint)", 
        "acceptance_rate": "",
        "acceptance_type": "",
        "citation": "<i>(Preprint)</i> <u>Abhishek Divekar</u>, Mudit Agarwal, Srujana Merugu, and Nikhil Rasiwasia. <b>\"Unsupervised text augmentation using Pre-trained Paraphrase Generation\"</b>.", 
        "excerpt": "Unsupervised text augmentation has gained attention in recent years, as approaches which use pre-trained models to produce high-quality augmentations (such as Backtranslation [48]) are replacing simple rule-based noising to attain SOTA performance in fully- and semi-supervised settings [53]. Such approaches have benefits over other model-based text augmentations, as they are applicable to most natural language tasks and their efficacy does not rely on the ava

## Import pandas

We are using the very handy pandas library for dataframes.

In [347]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [348]:
publications = pd.read_json("publications.json")
publications 

,title,pub_date,url_slug,paper_url,teaser_url,venue,acceptance_rate,acceptance_type,citation,excerpt,alternative_url
0,Unsupervised text augmentation using Pre-train...,2023-09-01,grok-paper,,,(Preprint),,,"<i>(Preprint)</i> <u>Abhishek Divekar</u>, Mud...",Unsupervised text augmentation has gained atte...,NaN
1,Squeezing the last DRiP: AutoML for cost-const...,2021-09-01,drip-paper,(internal),,Amazon Machine Learning Conference (AMLC),30%,Poster,"<u>Abhishek Divekar</u>*, Gaurav Manchanda*, P...",Recent progress in automated machine learning ...,"<a href=""https://adivekar-utexas.github.io/fil..."
2,CPP Multimodal AutoML Corpus and Benchmark,2021-09-01,cpp-benchmark-paper,(internal),,1st Workshop on MultiModal Learning and Fusion...,,Oral,"Andrew Borthwick, <u>Abhishek Divekar</u>, Nic...",A collection of 40 binary classification datas...,NaN
3,LEAP: LEAf node Predictions in the wild,2021-03-01,leap-paper,(internal),,2nd ASCS Applied Science Workshop,,Oral,"<u>Abhishek Divekar</u>, Vinayak Puranik, Zhen...",The data available in Amazon's catalog is rich...,NaN
4,"Entity Prediction Service: a configurable, end...",2020-09-01,eps-paper,(internal),,"Workshop on Automated Machine Learning, Amazon...",,Poster,"Gaurav Manchanda*, <u>Abhishek Divekar</u>*, A...",Business teams at Amazon often need to classif...,NaN
5,Benchmarking datasets for Anomaly-based Networ...,2018-10-25,kddcup99-paper,https://arxiv.org/abs/1811.05372,,"IEEE International Conference on Computing, Co...",,Oral,"<u>Abhishek Divekar</u>, Meet Parekh, Vaibhav ...",Machine Learning has been steadily gaining tra...,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [349]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [350]:
import os
for row, item in publications.iterrows():
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    md = "---\ntitle: \""   + item.title + '"'
    md += "\ncollection: publications"
    md += f'\nurlslug: "{item.url_slug}"'
    md += """\npermalink: /publication/""" + html_filename
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    md += "\ndate: " + str(item.pub_date) 
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    if len(str(item.acceptance_rate)) > 0:
        md += "\nacceptancerate: '" + item.acceptance_rate + "'"
    if len(str(item.acceptance_type)) > 0:
        md += "\nacceptancetype: '" + item.acceptance_type + "'"
    if len(str(item.alternative_url)) > 5:
        md += "\nalternativeurl: '" + item.alternative_url + "'"
    if len(str(item.teaser_url)) > 5:
        md += "\nteaserurl: '" + item.teaser_url + "'"
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    md += "\n---"
    
    ## Markdown description for individual page
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    if len(str(item.paper_url)) > 5 and "internal" not in str(item.paper_url):
        md += "\n[Download](" + item.paper_url + ")\n"  
    md += f"\nCitation: {item.citation} "
    if len(str(item.acceptance_type)) > 0:
        md += f"({item.acceptance_type}) "
    if len(str(item.acceptance_rate)) > 0:
        md += f"(acceptance-rate: {item.acceptance_rate}) "
    md_filename = os.path.basename(md_filename)
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [351]:
!ls ../_publications/

2018-10-25-kddcup99-paper.md      2021-09-01-cpp-benchmark-paper.md
2020-09-01-eps-paper.md           2021-09-01-drip-paper.md
2021-03-01-leap-paper.md          2023-09-01-grok-paper.md


In [352]:
!cat ../_publications/2021-03-01-leap-paper.md

---
title: "LEAP: LEAf node Predictions in the wild"
collection: publications
urlslug: "leap-paper"
permalink: /publication/2021-03-01-leap-paper
excerpt: 'The data available in Amazon&apos;s catalog is rich and diverse; however, it is also highly irregular and often challenging to employ directly for business or Machine Learning applications. Frequent issues include low fill-rate of catalog attributes, noise in attributes, dataset shift between train and real-world distributions, and potential abuse in externally-sourced fields such as Generic Keywords and Browse Node. In this paper, we work backward from the goal of building high-precision classifiers to predict “Leaf Nodes” of Amazon&apos;s Browse taxonomy, to address the issue of purposeful or accidental mis-noding in the face of aforementioned challenges. Our findings indicate that weakly-supervised datasets collected using intuitive filters - based on Glance Views (GVs) and Total Orders - are effective in eliminating potential no

In [353]:
from datetime import datetime
print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

2023-01-01 21:18:50
